In [9]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
import numpy as np

np.random.seed(1234)

URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [10]:
# SETUP EVALUATORS
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2602 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2227 (17.6%) Users that have less than 1 test interactions


In [11]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
import pickle


MAP_recommender_per_group = {}

collaborative_recommender_class = {
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "SLIM_ELASTIC": MultiThreadSLIM_SLIMElasticNetRecommender,
                                   "TOP_POP": TopPop,
                                    "MULT_VAE": MultVAERecommender_OptimizerMask
                                   }
collaborative_recommender_class.items()

recommender_object_dict_train = {}
recommender_object_dict_train_validation = {}
recommender_object_dict_all = {}
for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train)
    recommender_object_dict_train[label] = recommender_object
    
    recommender_object = recommender_class(URM_train_validation)
    recommender_object_dict_train_validation[label] = recommender_object
    
    recommender_object = recommender_class(URM_all)
    recommender_object_dict_all[label] = recommender_object
    
    
recommender_object_dict_train["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=8)
recommender_object_dict_train["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict_train["RP3beta"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict_train["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict_train["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
recommender_object_dict_train["TOP_POP"].fit()
recommender_object_dict_train["MULT_VAE"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)

recommender_object_dict_train_validation["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=8)
recommender_object_dict_train_validation["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict_train_validation["RP3beta"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict_train_validation["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict_train_validation["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
recommender_object_dict_train_validation["TOP_POP"].fit()
recommender_object_dict_train_validation["MULT_VAE"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)

recommender_object_dict_all["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=8)
recommender_object_dict_all["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict_all["RP3beta"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict_all["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict_all["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
recommender_object_dict_all["TOP_POP"].fit()
recommender_object_dict_all["MULT_VAE"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)



# save the trained dict objects
with open('recommender_object_dict_train.pickle', 'wb') as handle:
    pickle.dump(recommender_object_dict_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('recommender_object_dict_train_validation.pickle', 'wb') as handle:
    pickle.dump(recommender_object_dict_train_validation, handle, protocol=pickle.HIGHEST_PROTOCOL)

2023-12-17 17:04:38.838379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 17:04:38.838467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 17:04:38.893513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 17:04:39.010631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-17 17:04:40.315596: W tensorflow/compiler/tf2

UserKNNCFRecommender: URM Detected 455 ( 3.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 335 ( 1.5%) items with no interactions.
UserKNNCFRecommender: URM Detected 213 ( 1.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 117 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 455 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 335 ( 1.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 213 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 117 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 455 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 335 ( 1.5%) items with no interactions.
P3alphaRecommender: URM Detected 213 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 117 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 455 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 33

  2%|▏         | 488/22222 [00:04<01:55, 187.53it/s]Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:


KeyboardInterrupt: 

In [ ]:
# reload the trained dict objects
import pickle
with open('recommender_object_dict_train.pickle', 'rb') as handle:
    recommender_object_dict_train = pickle.load(handle)
with open('recommender_object_dict_train_validation.pickle', 'rb') as handle:
    recommender_object_dict_train_validation = pickle.load(handle)

In [41]:
import  Recommenders.XGBoostEnsemble as XgBoostEnsemble
from importlib import reload
reload(XgBoostEnsemble)

<module 'Recommenders.XGBoostEnsemble' from '/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/Recommenders/XGBoostEnsemble.py'>

In [4]:
# import numpy as np
# import scipy.sparse as sps
# import pandas as pd
# import sklearn.svm
# import tqdm
# import xgboost
# from sklearn.model_selection import train_test_split
# from xgboost import XGBRanker
# from Recommenders.BaseRecommender import BaseRecommender
# from xgboost import plot_importance
# 
# 
# class XgBoostEnsembler(BaseRecommender):
#     """
#     This class is used to create an ensemble model using XGBoost.
#     It inherits from the BaseRecommender class.
#     """
#     RECOMMENDER_NAME = "XgBoostEnsembler"
# 
#     def __init__(self, URM_train, URM_val, recommenders: dict, recommenders_generate_data: dict, internal_cutoff_xgboost=10, verbose=True):
#         """
#         Constructor for the XgBoostEnsembler class.
# 
#         Parameters:
#         URM_train (scipy.sparse matrix): The user-item matrix for training.
#         URM_val (scipy.sparse matrix): The user-item matrix for validation.
#         recommenders (dict): A dictionary of recommenders.
#         verbose (bool): A flag used to print detailed logs. Default is True.
#         """
#         super().__init__(URM_train, verbose=verbose)
#         self.URM_val = URM_val
#         self.recommenders = recommenders
#         self.XGB_model = self._init_XGB_model()
#         self.internal_cutoff_xgboost = internal_cutoff_xgboost
#         self.recommenders_generate_data = recommenders_generate_data
#         self.training_dataframe = None
# 
#     def _init_XGB_model(self):
#         """
#         Initializes the XGBoost model with predefined parameters.
# 
#         Returns:
#         XGBRanker: The initialized XGBoost model.
#         """
#         return XGBRanker(
#             objective='rank:pairwise',
#             n_estimators=50,
#             learning_rate=1e-1,
#             reg_alpha=1e-1,
#             reg_lambda=1e-1,
#             max_depth=5,
#             max_leaves=0,
#             grow_policy="depthwise",
#             booster="gbtree",
#             # eta=0.0506,
#             # booster="gbtree",
#             # colsample_bytree=0.6113704247857885,
#             # gamma=0.09822,
#             # min_child_weight=7.0,
#             # tree_method="hist",
#             verbosity=1,
#             # max_depth=3, learning_rate=0.1, n_estimators=100,
#             # silent=True, objective="rank:pairwise", booster='gbtree',
#             # n_jobs=-1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0,
#             # subsample=1, colsample_bytree=1, colsample_bylevel=1,
#             # reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
#             # base_score=0.5,
# 
#         )
#         return xgboost.XGBRanker()
#         # return sklearn.linear_model.Ridge()
# 
#     def _create_training_dataframe(self, users):
#         """
#         Creates a training dataframe with user IDs and their corresponding item recommendations.
# 
#         Parameters:
#         n_users (int): The number of users.
# 
#         Returns:
#         DataFrame: The training dataframe.
#         """
# 
#         print("Creating training dataframe...")
#         training_dataframe = pd.DataFrame(index=users, columns=["ItemID"])
#         training_dataframe["UserID"] = users
#         # training_dataframe.index.name = 'UserID'
# 
#         all_user_recommendations = np.zeros((len(users), self.internal_cutoff_xgboost * len(self.recommenders_generate_data)), dtype=np.int64)
# 
#         for i, (_, rec_instance) in enumerate(self.recommenders_generate_data.items()):
#             recs = rec_instance.recommend(users, cutoff=self.internal_cutoff_xgboost)
#             all_user_recommendations[:, i*self.internal_cutoff_xgboost:(i+1)*self.internal_cutoff_xgboost] = recs
# 
#         training_dataframe["ItemID"] = all_user_recommendations.tolist()
# 
#         return training_dataframe.explode("ItemID").drop_duplicates()
# 
#     def _populate_training_dataframe_label(self, training_dataframe):
#         print("Populating training dataframe with labels...")
#         urm_validation_coo = sps.coo_matrix(self.URM_val)
# 
#         correct_recommendations = pd.DataFrame({"UserID": urm_validation_coo.row,
#                                                 "ItemID": urm_validation_coo.col})
#         training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID', 'ItemID'], how='left',
#                                       indicator='Exist')
# 
#         training_dataframe["Label"] = training_dataframe["Exist"] == "both"
#         training_dataframe.drop(columns=['Exist'], inplace=True)
#         # training_dataframe = training_dataframe.set_index('UserID')
# 
#         print("Percentage of positive samples: {:.2f}%".format(
#             training_dataframe["Label"].sum() / training_dataframe.shape[0] * 100))
# 
#         return training_dataframe
# 
#     def _populate_training_dataframe(self, training_dataframe, users):
#         """
#         Populates the training dataframe with item scores from each recommender.
# 
#         Parameters:
#         training_dataframe (DataFrame): The training dataframe.
#         n_users (int): The number of users.
# 
#         Returns:
#         DataFrame: The populated training dataframe.
#         """
#         # for user_id in tqdm.tqdm(users):
#         #     for rec_label, rec_instance in self.recommenders.items():
#         #         item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
#         #         all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute=item_list)
#         #         training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]
# 
#         print("Populating training dataframe with item scores...")
#         for rec_label, rec_instance in tqdm.tqdm(self.recommenders.items()):
#             all_user_scores = rec_instance._compute_item_score(users)  # Predict items for all users at once
#             map_user_to_index = {user_id: index for index, user_id in enumerate(users)}
#             user_index_pairs = training_dataframe[["UserID", "ItemID"]].values.astype(np.int64)
#             users_indexes = [map_user_to_index[user_id] for user_id in user_index_pairs[:,0]]
# 
#             training_dataframe[rec_label] = all_user_scores[users_indexes, user_index_pairs[:, 1]]
# 
# 
#         item_popularity = np.ediff1d(sps.csc_matrix(self.URM_train).indptr)
#         training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
# 
#         user_popularity = np.ediff1d(sps.csr_matrix(self.URM_train).indptr)
#         training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
# 
#         training_dataframe = training_dataframe.sort_values("UserID").reset_index()
#         training_dataframe.drop(columns=['index'], inplace=True)
#         training_dataframe["ItemID"] = training_dataframe["ItemID"].astype('int64')
#         return training_dataframe
# 
#     def fit(self, plot=False, prepare_training_df=True):
#         """
#         Trains the XGBoost model.
#         """
#         self.XGB_model = self._init_XGB_model()
#         if prepare_training_df or self.training_dataframe is None:
#             n_users, n_items = self.URM_train.shape
#             training_dataframe = self._create_training_dataframe(range(n_users))
#             training_dataframe = self._populate_training_dataframe_label(training_dataframe)
#             self.training_dataframe = self._populate_training_dataframe(training_dataframe, range(n_users))
# 
#         X = self.training_dataframe.drop(columns=["Label"])
#         y = self.training_dataframe["Label"]
#         groups = self.training_dataframe.groupby("UserID").size().values
# 
#         # Splitting X, y into train and validation sets
#         X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=False)
# 
#         # Creating groups for train and validation sets
#         groups_train = X_train.groupby("UserID").size().values
#         groups_val = X_val.groupby("UserID").size().values
# 
#         # X_train = X_train.drop(columns=["UserID", "ItemID"])
#         # X_val = X_val.drop(columns=["UserID", "ItemID"])
# 
#         # evals = [(X_train, y_train), (X_val, y_val)]
# 
#         # groups = training_dataframe.groupby("UserID").size().values
#         # self.XGB_model.fit(X_train, y_train, group=groups, verbose=True, eval_set=[(X_test, y_test)], eval_group=groups, early_stopping_rounds=10, eval_metric=["auc","error"])
#         # self.XGB_model.fit(
#         #     X_train, y_train, group=groups_train, verbose=True, eval_set=evals, eval_group=[groups_train, groups_val], early_stopping_rounds=10,
#         #      eval_metric=['map@10']
#         # )
#         # self.XGB_model.fit(
#         #     X_train, y_train,
#         #     # verbose=True, #eval_set=evals,
#         #     # early_stopping_rounds=10,
#         #     # eval_metric=['merror']
#         # )
#         self.XGB_model.fit(X_train, y_train, group=groups_train, verbose=True)
#         y_est = self.XGB_model.predict(X_val)
#         # MAP
#         print("MSE: {:.2f}".format(np.mean((y_val - y_est) ** 2)))
# 
#         # self.XGB_model.fit(X_train, y_val)
#         if plot:
#             plot_importance(self.XGB_model, importance_type='weight', title='Weight (Frequence)')
# 
#     def _compute_item_score(self, user_id_array, items_to_compute=None):
#         """
#         Computes the item scores for a given array of user IDs.
# 
#         Parameters:
#         user_id_array (array): The array of user IDs.
#         items_to_compute (array): The array of items to compute scores for. Default is None.
# 
#         Returns:
#         array: The array of item scores.
#         """
#         n_users, n_items = self.URM_train.shape
#         training_dataframe = self._create_training_dataframe(user_id_array)
#         training_dataframe = self._populate_training_dataframe(training_dataframe, user_id_array)
#         X_train = training_dataframe
#         # X_train = training_dataframe.drop(columns=["UserID", "ItemID"])
#         predictions = self.XGB_model.predict(X_train)
#         return self._rerank_items(predictions, training_dataframe, user_id_array, n_items)
# 
#     def _rerank_items(self, predictions, training_dataframe, user_id_array, n_items):
#         """
#         Reranks the items based on the predictions from the XGBoost model.
# 
#         Parameters:
#         predictions (array): The array of predictions from the XGBoost model.
#         training_dataframe (DataFrame): The training dataframe.
#         user_id_array (array): The array of user IDs.
#         n_items (int): The number of items.
# 
#         Returns:
#         array: The array of reranked item scores.
#         """
#         reranked_dataframe = training_dataframe.copy()
#         reranked_dataframe['rating_xgb'] = pd.Series(predictions, index=reranked_dataframe.index)
#         reranked_dataframe = reranked_dataframe.sort_values(['UserID', 'rating_xgb'], ascending=[True, False])
#         result = []
#         for id in tqdm.tqdm(user_id_array):
#             items_scores = np.ones(n_items) * (-100000000)
#             items_scores[reranked_dataframe.loc[reranked_dataframe['UserID'] == id].ItemID.values.astype(int)] = \
#                 reranked_dataframe.loc[reranked_dataframe['UserID'] == id].rating_xgb.values
#             result.append(items_scores)
#         return np.array(result)


## TRY to use the XGBoostEnsemble

In [5]:
# import  Recommenders.XGBoostEnsemble as XgBoostEnsemble
# from importlib import reload
# reload(XgBoostEnsemble)

xgb_ranker_params = {
    "objective": "rank:pairwise",
    "n_estimators": 50,
    "learning_rate": 1e-1,
    "reg_alpha": 1e-1,
    "reg_lambda": 1e-1,
    "max_depth": 5,
    "max_leaves": 0,
    "grow_policy": "depthwise",
    "booster": "gbtree",
    "verbosity": 1,
}
model = XgBoostEnsembler(URM_train, URM_val=URM_validation, recommenders=recommender_object_dict_train, recommenders_generate_data=recommender_object_dict_train)

model.fit(**xgb_ranker_params)

XgBoostEnsembler: URM Detected 455 ( 3.6%) users with no interactions.
XgBoostEnsembler: URM Detected 335 ( 1.5%) items with no interactions.
Creating training dataframe...


  0%|          | 0/6 [00:00<?, ?it/s]

Populating training dataframe with labels...
Percentage of positive samples: 3.07%
Populating training dataframe with item scores...


100%|██████████| 6/6 [00:09<00:00,  1.62s/it]


MSE: 0.41


In [6]:
model.recommenders_generate_data = recommender_object_dict_train_validation
model.recommenders = recommender_object_dict_train_validation
evaluator_test.evaluateRecommender(model)

Creating training dataframe...


  0%|          | 0/6 [00:00<?, ?it/s]

Populating training dataframe with item scores...


100%|██████████| 1000/1000 [00:01<00:00, 596.87it/s]


Creating training dataframe...


...

 17%|█▋        | 1/6 [00:00<00:00,  9.02it/s]

Populating training dataframe with item scores...


100%|██████████| 411/411 [00:00<00:00, 564.44it/s]


EvaluatorHoldout: Processed 10411 (100.0%) in 51.49 sec. Users per second: 202


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.099491                 0.175275  0.148429  0.050748    0.087589   
 
             MRR      NDCG       F1  HIT_RATE ARHR_ALL_HITS  ... COVERAGE_USER  \
 cutoff                                                      ...                 
 10      0.27906  0.157536  0.11913  0.539622      0.371389  ...      0.823785   
 
        COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI SHANNON_ENTROPY  \
 cutoff                                                                
 10              0.444532    0.823785        0.04774        9.778185   
 
        RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY  \
 cutoff                                                                         
 10                       0.995834             0.138732              0.753987   
 
        RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cut

In [8]:
#evaluate recommenders
for label, recommender in recommender_object_dict_train_validation.items():
    print("Evaluating recommender: {}".format(label))
    result_dict, _ = evaluator_test.evaluateRecommender(recommender)
    print(result_dict["MAP"].item())

Evaluating recommender: UserKNNCF
EvaluatorHoldout: Processed 10411 (100.0%) in 8.81 sec. Users per second: 1182
0.03582347654266798
Evaluating recommender: ItemKNNCF
EvaluatorHoldout: Processed 10411 (100.0%) in 7.40 sec. Users per second: 1407
0.04596670493510432
Evaluating recommender: P3alpha
EvaluatorHoldout: Processed 10411 (100.0%) in 5.69 sec. Users per second: 1831
0.04764005333187027
Evaluating recommender: RP3beta
EvaluatorHoldout: Processed 10411 (100.0%) in 5.87 sec. Users per second: 1775
0.04749910808622816
Evaluating recommender: SLIM_ELASTIC
EvaluatorHoldout: Processed 10411 (100.0%) in 7.87 sec. Users per second: 1322
0.05021803480140885
Evaluating recommender: TOP_POP
EvaluatorHoldout: Processed 10411 (100.0%) in 5.65 sec. Users per second: 1842
0.012087829112370427


In [17]:
from xgboost import XGBRanker
from Recommenders.XGBoostEnsemble import XgBoostEnsembler

import optuna as op
import xgboost as xgb

model2 = XgBoostEnsembler(URM_train, URM_val=URM_validation, recommenders=)

model2.training_dataframe = model.training_dataframe

def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "rank:pairwise",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear"]),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    # xgb_model = XGBRanker(**param)
    # XGBRanker(
    #     objective='rank:pairwise',
    #     n_estimators=1000,
    #     learning_rate=0.19823429576094637,
    #     reg_alpha=47,
    #     reg_lambda=0.313,
    #     max_depth=3,
    #     verbosity=0,
    #     booster="gbtree",
    #     colsample_bytree=0.6113704247857885,
    #     gamma=8.964184693722684,
    #     min_child_weight=7.0,
    #     tree_method="hist"
    # )
    # Add a callback for pruning.
    #pruning_callback = op.integration.XGBoostPruningCallback(trial, "validation-auc")
    model.fit(prepare_training_df=False)
        
    result, _ = evaluator_test.evaluateRecommender(model)
    MAP_result = result["MAP"].item()
       
    return MAP_result

XgBoostEnsembler: URM Detected 429 ( 3.4%) users with no interactions.
XgBoostEnsembler: URM Detected 333 ( 1.5%) items with no interactions.


In [ ]:
best_params = {
    
}

study_name = "XGB-study-2"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize",pruner=op.pruners.MedianPruner(n_warmup_steps=5), load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=10)

In [48]:
import  Recommenders.XGBoostEnsemble as XgBoostEnsemble
from importlib import reload
reload(XgBoostEnsemble)
model = XgBoostEnsemble.XgBoostEnsembler(URM_train, URM_val=URM_validation, recommenders=recommender_object_dict)
model.training_dataframe = td_f
model.fit(prepare_training_df=False)

XgBoostEnsembler: URM Detected 429 ( 3.4%) users with no interactions.
XgBoostEnsembler: URM Detected 333 ( 1.5%) items with no interactions.


In [49]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(model)

EvaluatorHoldout: Ignoring 2162 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users



100%|██████████| 6/6 [00:03<00:00,  1.90it/s]

100%|██████████| 1000/1000 [00:01<00:00, 577.30it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.92it/s]

100%|██████████| 1000/1000 [00:01<00:00, 583.14it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.91it/s]

100%|██████████| 1000/1000 [00:01<00:00, 575.42it/s]

100%|██████████| 6/6 [00:03<00:00,  1.64it/s]

100%|██████████| 1000/1000 [00:01<00:00, 538.33it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.82it/s]

100%|██████████| 1000/1000 [00:01<00:00, 582.61it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.88it/s]

100%|██████████| 1000/1000 [00:01<00:00, 628.87it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.94it/s]

100%|██████████| 1000/1000 [00:01<00:00, 636.67it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.60it/s]

100%|██████████| 1000/1000 [00:01<00:00, 546.13it/s][A

100%|██████████| 6/6 [00:03<00:00,  1.94it/s]

100%|██████████| 1000/1000 [00:01<00:00, 573.88it/s][A

100%|██████████| 6/6 [00:02<00:00,  2.02it/s]

100%|██████████| 1000/1000 

EvaluatorHoldout: Processed 10476 (100.0%) in 1.11 min. Users per second: 157


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.079553                 0.139499  0.118535  0.038665    0.067016   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.233659  0.125583  0.095209  0.475372      0.298202  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.828929           0.39405    0.828929       0.047282   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10             9.78825                   0.996091             0.137432   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [44]:
#final.fit(alphas=[3,1.25])
evaluator_test.evaluateRecommender(recoslimmmender_object_dict["SLIM_ELASTIC"])

EvaluatorHoldout: Processed 10476 (100.0%) in 7.03 sec. Users per second: 1490


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.077491                 0.135103  0.114663  0.037391    0.064487   
 
             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.22838  0.121802  0.092482  0.468977      0.290177  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.828929          0.388748    0.828929       0.049943   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.010709                   0.997321             0.145165   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [45]:
td_f = model.training_dataframe